In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
len(objectIDs)

474632

In [5]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id, 
                        'url': url,
                        'filename': filename
                        })

In [6]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [7]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [8]:
images_saved = []
met_objects = []

start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

In [9]:
start = datetime.now()

for i, object_id in enumerate(objectIDs[340000:360000]):
    if i % 1000 == 0:
        with open(data_path + "images_saved_340.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects_340.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i + 340000, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass

340000: 0:00:00.006996 elapsed
341000: 0:15:38.222324 elapsed
342000: 0:33:09.869994 elapsed
343000: 0:50:17.135108 elapsed
344000: 1:06:23.770063 elapsed
345000: 1:21:48.794722 elapsed
346000: 1:40:49.693803 elapsed
347000: 1:56:12.321694 elapsed
348000: 2:11:43.984356 elapsed
349000: 2:31:12.830502 elapsed
350000: 2:46:42.786982 elapsed
351000: 3:02:08.514918 elapsed
352000: 3:21:35.730704 elapsed
353000: 3:37:31.869490 elapsed
354000: 3:55:31.803686 elapsed
355000: 4:13:14.041079 elapsed
356000: 4:30:06.678623 elapsed
357000: 4:48:40.889546 elapsed
358000: 5:03:49.984412 elapsed
359000: 5:23:08.965037 elapsed


In [10]:
with open(data_path + "images_saved_340.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects_340.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [11]:
len(images_saved)

9053